In [1]:
# installs
!pip install -q pytorch-lightning==1.8.6

In [2]:
!git clone https://github.com/mk-minchul/AdaFace
%cd AdaFace
!ls

fatal: destination path 'AdaFace' already exists and is not an empty directory.
/content/AdaFace
assets	    dataset	       img1.jpg      net.py	  README_TRAIN.md   utils.py
config.py   evaluate_utils.py  inference.py  pretrained   requirements.txt  validation_hq
convert.py  face_alignment     LICENSE	     __pycache__  scripts	    validation_lq
data.py     head.py	       main.py	     README.md	  train_val.py	    validation_mixed


In [3]:
# imports
import os
import net
import torch
import gdown
import numpy as np
from face_alignment import align
from inference import load_pretrained_model, to_input

In [4]:
# download model
!mkdir -p pretrained
url = "https://drive.google.com/uc?id=1eUaSHG4pGlIZK7hBkqjyp2fc2epKoBvI"
output = "pretrained/adaface_ir50_ms1mv2.ckpt"
gdown.download(url, output)

Downloading...
From (original): https://drive.google.com/uc?id=1eUaSHG4pGlIZK7hBkqjyp2fc2epKoBvI
From (redirected): https://drive.google.com/uc?id=1eUaSHG4pGlIZK7hBkqjyp2fc2epKoBvI&confirm=t&uuid=6f739219-45a9-454c-81f8-098c8d490b60
To: /content/AdaFace/pretrained/adaface_ir50_ms1mv2.ckpt
100%|██████████| 700M/700M [00:04<00:00, 145MB/s]


'pretrained/adaface_ir50_ms1mv2.ckpt'

In [5]:
adaface_models = {
    'ir_50':"pretrained/adaface_ir50_ms1mv2.ckpt",
}

def load_pretrained_model(architecture='ir_50'):
    # load model and pretrained statedict
    assert architecture in adaface_models.keys()
    model = net.build_model(architecture)
    statedict = torch.load(adaface_models[architecture])['state_dict']
    model_statedict = {key[6:]:val for key, val in statedict.items() if key.startswith('model.')}
    model.load_state_dict(model_statedict)
    model.eval()
    return model

def to_input(pil_rgb_image):
    np_img = np.array(pil_rgb_image)
    brg_img = ((np_img[:,:,::-1] / 255.) - 0.5) / 0.5
    tensor = torch.tensor([brg_img.transpose(2,0,1)]).float()
    return tensor

In [6]:
if __name__ == '__main__':

    model = load_pretrained_model('ir_50')
    feature, norm = model(torch.randn(2,3,112,112))
    # add some images here and there
    test_image_path = '/content/images'
    features = []
    for fname in sorted(os.listdir(test_image_path)):
        path = os.path.join(test_image_path, fname)
        aligned_rgb_img = align.get_aligned_face(path)
        bgr_tensor_input = to_input(aligned_rgb_img)
        feature, _ = model(bgr_tensor_input)
        features.append(feature)

    similarity_scores = torch.cat(features) @ torch.cat(features).T
    print(similarity_scores)

/content/AdaFace/face_alignment/mtcnn_pytorch/src/matlab_cp2tform.py:90: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  r, _, _, _ = lstsq(X, U)
<ipython-input-5-7bdc3490688e>:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  tensor = torch.tensor([brg_img.transpose(2,0,1)]).float()


tensor([[1.0000, 0.5145],
        [0.5145, 1.0000]], grad_fn=<MmBackward0>)
